In [2]:
import spacy
from spacy import displacy
from spacy.pipeline import DependencyParser
from collections import Counter
# import en_core_web_sm
from pprint import pprint
nlp = spacy.load('en_core_web_sm')

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from P

In [3]:
sampleText = u"McCoy has now been held below 10 touches in 2-of-3 games and has handled the ball only 29 times all season."
# sampleText = "Mark Ingram returned from his four-week suspension to make his 2018 debut and made the start for the Saints."
# sampleText = "Kamara got the start and out-touched Mark Ingram 19-14 one game after Ingram out-touched Kamara 18-9 in his 2018 debut. Both players received goal line looks, but only Kamara cashed his in."

In [4]:
# Another sample
doc = nlp('European authorities fined Google a record $5.1 billion on' +
          'Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
pprint([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'),
 ('Google', 'ORG'),
 ('$5.1 billion', 'MONEY'),
 ('onWednesday', 'DATE')]


In [5]:
doc = nlp(sampleText)
pprint([(X.text, X.label_) for X in doc.ents])

[('McCoy', 'ORG'),
 ('10', 'CARDINAL'),
 ('2-of-3', 'CARDINAL'),
 ('only 29', 'CARDINAL'),
 ('all season', 'DATE')]


In [6]:
# pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

In [7]:
# NER
displacy.render(nlp(sampleText), jupyter=True, style='ent')
dict([(str(x), x.label_) for x in nlp(sampleText).ents])

{'10': 'CARDINAL',
 '2-of-3': 'CARDINAL',
 'McCoy': 'ORG',
 'all season': 'DATE',
 'only 29': 'CARDINAL'}

In [8]:
for token in doc:
    print(token.text, token.i, token.pos_, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

McCoy 0 PROPN nsubjpass held VERB []
has 1 VERB aux held VERB []
now 2 ADV advmod held VERB []
been 3 VERB auxpass held VERB []
held 4 VERB ROOT held VERB [McCoy, has, now, been, below, and, handled, .]
below 5 ADP prep held VERB [touches]
10 6 NUM nummod touches NOUN []
touches 7 NOUN pobj below ADP [10, in]
in 8 ADP prep touches NOUN [2-of-3, games]
2-of-3 9 NUM punct in ADP []
games 10 NOUN pobj in ADP []
and 11 CCONJ cc held VERB []
has 12 VERB aux handled VERB []
handled 13 VERB conj held VERB [has, ball, season]
the 14 DET det ball NOUN []
ball 15 NOUN dobj handled VERB [the]
only 16 ADV advmod 29 NUM []
29 17 NUM nummod season NOUN [only, times]
times 18 NOUN quantmod 29 NUM []
all 19 DET det season NOUN []
season 20 NOUN npadvmod handled VERB [29, all]
. 21 PUNCT punct held VERB []


In [9]:
# If CCONJ has cc dep on PROPN's nsubjpass, break sentence as:
subSentences = {}
subSentenceCounter = 0
subSentences[subSentenceCounter] = []
subject = None
propn = None

for token in doc:
    if token.pos_ == 'PROPN' and (token.dep_ == 'nsubjpass' or token.dep_ == 'nsubj'):
        subject = token.head.text
        propn = token.text
        
    if token.pos_ == 'CCONJ' and token.dep_ == 'cc' and token.head.text == subject:
        subSentences[subSentenceCounter].append('.')
        subSentenceCounter += 1
        subSentences[subSentenceCounter] = []
        subSentences[subSentenceCounter].append(propn)
    else:
        subSentences[subSentenceCounter].append(token.text)
        
for k,v in subSentences.items():
    print(" ".join(v).replace(' .', '.').replace(' - ', '-').replace(' ,', ','))

McCoy has now been held below 10 touches in 2-of-3 games.
McCoy has handled the ball only 29 times all season.


In [10]:
# Dependencies graph
displacy.render(nlp(sampleText), style='dep', jupyter = True, options = {'distance': 120})

In [11]:
# POS and lemmatizing
[(x.orth_,x.pos_, x.lemma_) for x in [y for y in nlp(sampleText) if not y.is_stop and y.pos_ != 'PUNCT']]

[('McCoy', 'PROPN', 'mccoy'),
 ('held', 'VERB', 'hold'),
 ('10', 'NUM', '10'),
 ('touches', 'NOUN', 'touch'),
 ('2-of-3', 'NUM', '2-of-3'),
 ('games', 'NOUN', 'game'),
 ('handled', 'VERB', 'handle'),
 ('ball', 'NOUN', 'ball'),
 ('29', 'NUM', '29'),
 ('times', 'NOUN', 'time'),
 ('season', 'NOUN', 'season')]

In [12]:
import csv
import numpy as np

In [18]:
# Load file to be chunked
reports = []

with open('../../../analysis/data/nbmodel_templates.csv', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            reports.append(row[1])
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are reports, templates
Processed 2884 lines.


In [19]:
reports[0]

'${player_name} caught ${receptions} passes for ${rec_yards} yards and a touchdown in week ${week} against the ${opp}.'

In [20]:
# Chunk sentences by prepositions
def chunkText(text):
    retval = []
    subSentences = {}
    subSentenceCounter = 0
    subSentences[subSentenceCounter] = []
    textDoc = nlp(text)
    
    for token in textDoc:
        if token.dep_ == 'prep':
    #         subSentences[subSentenceCounter].append('.')
            subSentenceCounter += 1
            subSentences[subSentenceCounter] = []
        subSentences[subSentenceCounter].append(token.text)

    for k,v in subSentences.items():
        retval.append(" ".join(v).replace(' .', '').replace(" '", "'").replace(" ’", "'").replace('$ { ', '${').replace(' }', '}'))
    
    return retval

In [22]:
# Write chunks to file
with open('../../../analysis/data/output_chunked.csv', mode='w') as chunk_file:
    chunk_writer = csv.writer(chunk_file, delimiter=',', lineterminator='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for text in reports:
        chunks = chunkText(text)
        chunk_writer.writerow(chunks)


In [21]:
for text in reports[:2]:
    chunks = chunkText(text)
    print(chunks)

['${player_name} caught ${receptions} passes', 'for ${rec_yards} yards and a touchdown', 'in week ${week}', 'against the ${opp}.']
['${player_name} caught ${receptions} passes', 'for ${rec_yards} yards and a touchdown', 'in week ${week}', 'against the ${opp}.']
